In [1]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import pickle
import time
openai.api_key = 'YOUR_API_KEY'  # Replace with your OpenAI API key

In [2]:
# Explaining the function that generates embeddings for a list of input texts using the OpenAI API.
# Explicando la función que genera los embeddings para una lista de textos de entrada utilizando la API de OpenAI.

def get_embeddings(texts):
    '''
    Generates embeddings for a list of input texts using the OpenAI API, handling token limits, rate limits, and connection timeouts.

    Parameters:
    -----------
    texts : list of str
        A list of textual data (documents, sentences, etc.) to generate embeddings for.

    Returns:
    --------
    np.array
        A NumPy array containing the embeddings for each input text.

    Functionality:
    --------------
    1. Splits input texts into smaller chunks based on the specified token limit (`max_tokens`).
    2. Sends each chunk to the OpenAI API to get embeddings, using the model specified (`text-embedding-3-small`).
    3. Handles potential errors:
        - `RateLimitError`: If the API rate limit is exceeded, waits for a specified time (`rate_limit_wait_time`) and retries.
        - `APIConnectionError`: If there is a connection issue, waits for a specified time (`timeout_wait_time`) and retries.
        - Other exceptions are caught and raised.
    4. Combines the embeddings from all chunks into a single NumPy array and returns it.

    Error Handling:
    ---------------
    - RateLimitError: Prints a message and waits before retrying.
    - APIConnectionError: Prints a message and waits before retrying.
    - Any other error is raised after printing the error message.
    '''

    max_tokens = 8192  # Define max tokens per request
    rate_limit_wait_time = 60  # Time to wait in seconds if rate limit is hit
    timeout_wait_time = 20  # Time to wait in seconds if a timeout occurs


    def split_texts(texts, max_tokens):
        chunks = []
        current_chunk = []
        current_length = 0
        for text in texts:
            text_length = len(text.split())  # Word count
            if current_length + text_length > max_tokens:
                chunks.append(current_chunk)
                current_chunk = [text]
                current_length = text_length
            else:
                current_chunk.append(text)
                current_length += text_length
        if current_chunk:
            chunks.append(current_chunk)
        return chunks
    

    def get_embeddings_for_chunk(chunk):
        while True:
            try:
                response = openai.Embedding.create(
                    model="text-embedding-3-small",  # Choose appropriate model
                    input=chunk
                )
                return [data['embedding'] for data in response['data']]
            except openai.error.RateLimitError:
                print(f"Rate limit exceeded. Waiting for {rate_limit_wait_time} seconds...")
                time.sleep(rate_limit_wait_time)
            except openai.error.APIConnectionError:
                print(f"Connection timed out. Waiting for {timeout_wait_time} seconds...")
                time.sleep(timeout_wait_time)
            except Exception as e:
                print(f"An error occurred: {e}")
                raise


    # Split texts into chunks and get embeddings
    chunks = split_texts(texts, max_tokens)
    embeddings = []
    for chunk in chunks:
        embeddings.extend(get_embeddings_for_chunk(chunk))

    return np.array(embeddings)

In [4]:
embeddings_path = r'corpus_ccd_embeddings_full.pkl' # Load your embeddings here

print("Loading stored embeddings and metadata...")
with open(embeddings_path, 'rb') as file:
    data = pickle.load(file)
    summary_embeddings = data['summary_embeddings']
    saved_metadata = data['metadata']
    saved_summaries = data['summaries']

Loading stored embeddings and metadata...


In [8]:
def find_relevant_summaries(query, top_k=10):
    '''
    Finds the most relevant summaries for a given query by computing cosine similarity between the query embedding and precomputed summary embeddings.

    Parameters:
    -----------
    query : str
        The query text for which relevant summaries need to be found.
    
    top_k : int, optional (default=8)
        The number of top relevant summaries to return based on cosine similarity.

    Returns:
    --------
    list of int
        A list of indices corresponding to the top `top_k` most relevant summaries, sorted by descending cosine similarity.

    Functionality:
    --------------
    1. Converts the input query into an embedding using the `get_embeddings` function.
    2. Computes cosine similarities between the query embedding and the precomputed summary embeddings (`summary_embeddings`).
    3. Sorts the similarities in descending order and selects the top `top_k` most relevant summaries.
    4. Returns the indices of the most relevant summaries.

    Notes:
    ------
    - Assumes that `summary_embeddings` is a global variable containing the embeddings for the summaries.
    - The higher the cosine similarity, the more relevant the summary is to the query.
    '''
    query_embedding = get_embeddings([query])[0]
    cos_similarities = cosine_similarity([query_embedding], summary_embeddings)[0]
    top_indices = cos_similarities.argsort()[-top_k:][::-1]
    return top_indices

In [13]:
# Example query
query = input("")

# Retrieve relevant summaries
top_indices = find_relevant_summaries(query)

# Prepare the relevant information from summaries and texts
relevant_info = []
for idx in top_indices:
    summary = saved_summaries[idx]
    row_metadata = saved_metadata.iloc[idx]

    relevant_info.append(f"Resolución: {row_metadata['Número de Resolución']}\nFecha de Resolución:{row_metadata['Fecha de Resolución']}\nHechos: {summary}\nEnlace: {row_metadata['Enlace']}")


# Combine relevant information into a single prompt for the LLM
combined_info = "\n\n".join(relevant_info)

def generate_response(prompt):
    '''
    Generates a legal analysis response based on a given prompt using the OpenAI basic GPT-4o-mini for legal tasks.

    Parameters:
    -----------
    prompt : str
        The input text or query for which the response will be generated. This could be a description of a legal case or a set of facts for which jurisprudence is being analyzed.

    Returns:
    --------
    str
        The generated response containing the legal analysis, relevant facts, and a conclusion based on the input prompt.

    Functionality:
    --------------
    1. Sends a request to the OpenAI API using the `gpt-4o` model, with the system role providing specific instructions for legal drafting.
    2. The system prompt instructs the model to:
        - Identify the relevant facts and criteria from jurisprudence similar to the case.
        - Conduct a legal analysis and offer a conclusion.
    3. The user's input (`prompt`) is passed as the main content for the model to process.
    4. The response is generated with a maximum of 1200 tokens and a temperature of 0.8 for balanced creativity and coherence.
    5. Returns the content of the model's response.

    Notes:
    ------
    - The model used, `gpt-4o-mini`, is instructed to identify relevant jurisprudence and provide a legal conclusion.
    - The `temperature=0.8` ensures that the response is somewhat creative while remaining grounded in legal reasoning.
    '''

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Eres un asistente experto en redacción legal. A partir de la Jurisprudencia Relevante, vas a identificar cuáles son los hechos y criterios de la Jurisprudencia Relevante que son más similares a los Hechos del Caso. Después de identificar, debes precisar el análisis jurídico pertinente y ofrecer una conclusión"},
            
            {"role": "user", "content": prompt}
        ],
        max_tokens=1200,
        temperature=0.7
    )
    return response.choices[0].message['content']

# Create the prompt with the relevant information and query
prompt = f"Hechos del Caso:\n\n {query} \n\n Jurisprudencia Relevante:\n\n{combined_info}"
response = generate_response(prompt)

# Output the response
print(f"Hechos del Caso: {query}\n\nInforme: {response}\n\nJurisprudencia Relevante: \n\n{combined_info}")

Hechos del Caso: Una municipalidad ha estado ofreciendo el servicio de veterinaria a los vecinos. Este consiste en desparasitación, peluquería y corte de uñas. ¿Es un acto de violación de normas?

Informe: ### Identificación de Hechos y Criterios Similares

#### Hechos del Caso:
- Una municipalidad ofrece servicios veterinarios a los vecinos, incluyendo desparasitación, peluquería y corte de uñas.
- Se pregunta si estos actos constituyen una violación de normas.

#### Jurisprudencia Relevante:
1. **Resolución 94-2022-CCD-INDECOPI-CUS**:
   - Se denuncia a la Municipalidad Distrital de San Jerónimo por ofrecer servicios veterinarios (esterilización, consultas, desparasitaciones) supuestamente en competencia con servicios privados.
   - La comisión concluye que estos servicios son preventivos y no empresariales, por lo que no hay violación de normas.
   - Se recomienda evaluar la autorización de funcionamiento, pero no se imponen sanciones.

2. **Resolución 144-2018-CCD**:
   - Se distin